git clone https://github.com/Rudrabha/Wav2Lip.git
cd Wav2Lip

pip install dataclasses
pip install opencv-contrib-python==4.5.5.64
pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio==0.7.2 -f https://download.pytorch.org/whl/cpu/torch_stable.html
pip install -r requirements.txt --no-deps
pip list :
     (echolip) PS E:\MINI_0.1\Wav2Lip> pip list
Package               Version
--------------------- ---------
certifi               2021.5.30
dataclasses           0.8
numpy                 1.19.5
opencv-contrib-python 4.5.5.64
Pillow                8.4.0
pip                   21.2.2
setuptools            58.0.4
torch                 1.7.1+cpu
torchaudio            0.7.2
torchvision           0.8.2+cpu
typing_extensions     4.1.1
wheel                 0.37.1
wincertstore          0.2
(echolip) PS E:\MINI_0.1\Wav2Lip> 
> pip install face-detection==0.2.1

conda install -c conda-forge ffmpeg
"E:\MINI_0.1\Wav2Lip\face_detection\detection\sfd\s3fd.pth"(Install this from goofle or kgle and rename the path)
pip install tqdm==4.45.0 numba==0.48
librosa==0.7.0

comment for training :
(echolip) PS E:\MINI_0.1\Wav2Lip> python preprocess.py --data_root E:\MINI_0.1\input_videos_mp4_only --preprocessed_root E:\MINI_0.1\preprocessed_data --ngpu 0



In [ ]:
import sys

if sys.version_info[0] < 3 and sys.version_info[1] < 2:
    raise Exception("Must be using >= Python 3.2")

from os import listdir, path

if not path.isfile('face_detection/detection/sfd/s3fd.pth'):
    raise FileNotFoundError('Save the s3fd model to face_detection/detection/sfd/s3fd.pth \
                            before running this script!')

import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import argparse, os, cv2, traceback, subprocess
from tqdm import tqdm
from glob import glob
import audio
from hparams import hparams as hp
import torch  # added to check CUDA availability

import face_detection

parser = argparse.ArgumentParser()

parser.add_argument('--ngpu', help='Number of GPUs across which to run in parallel', default=1, type=int)
parser.add_argument('--batch_size', help='Single GPU Face detection batch size', default=32, type=int)
parser.add_argument("--data_root", help="Root folder of the LRS2 dataset", required=True)
parser.add_argument("--preprocessed_root", help="Root folder of the preprocessed dataset", required=True)

args = parser.parse_args()

# SAFER: check for CUDA availability
if args.ngpu > 0 and not torch.cuda.is_available():
    print(" Warning: CUDA not available! Falling back to CPU.")
    args.ngpu = 0

# Device list preparation
device_list = ['cuda:{}'.format(i) for i in range(args.ngpu)] if args.ngpu > 0 else ['cpu']

# Initialize FaceAlignment for each device
fa = [face_detection.FaceAlignment(face_detection.LandmarksType._2D, flip_input=False, device=device) for device in device_list]

# FFMPEG command template
template = 'ffmpeg -loglevel panic -y -i "{}" -strict -2 "{}"'

# Function to process video frames
def process_video_file(vfile, args, gpu_id):
    video_stream = cv2.VideoCapture(vfile)

    frames = []
    while 1:
        still_reading, frame = video_stream.read()
        if not still_reading:
            video_stream.release()
            break
        frames.append(frame)

    vidname = os.path.basename(vfile).split('.')[0]
    dirname = os.path.basename(os.path.dirname(vfile))  # safer path extraction

    fulldir = path.join(args.preprocessed_root, dirname, vidname)
    os.makedirs(fulldir, exist_ok=True)

    batches = [frames[i:i + args.batch_size] for i in range(0, len(frames), args.batch_size)]

    i = -1
    for fb in batches:
        preds = fa[gpu_id].get_detections_for_batch(np.asarray(fb))

        for j, f in enumerate(preds):
            i += 1
            if f is None:
                continue

            x1, y1, x2, y2 = f
            cv2.imwrite(path.join(fulldir, '{}.jpg'.format(i)), fb[j][y1:y2, x1:x2])

# Function to process audio extraction
def process_audio_file(vfile, args):
    vidname = os.path.basename(vfile).split('.')[0]
    dirname = os.path.basename(os.path.dirname(vfile))

    fulldir = path.join(args.preprocessed_root, dirname, vidname)
    os.makedirs(fulldir, exist_ok=True)

    wavpath = path.join(fulldir, 'audio.wav')

    command = template.format(vfile, wavpath)
    subprocess.call(command, shell=True)

# Multiprocessing handler
def mp_handler(job):
    vfile, args, gpu_id = job
    try:
        process_video_file(vfile, args, gpu_id)
    except KeyboardInterrupt:
        exit(0)
    except:
        traceback.print_exc()

# MAIN FUNCTION
def main(args):
    print('Started processing for {} with {} GPU(s)'.format(args.data_root, args.ngpu))

    # Match both "*/video.mp4" and also "video.mp4" if flat folder (your case)
    filelist = glob(path.join(args.data_root, '*.mp4')) + glob(path.join(args.data_root, '*/*.mp4'))

    if len(filelist) == 0:
        print(" No MP4 files found in {}".format(args.data_root))
        return

    gpu_count = args.ngpu if args.ngpu > 0 else 1
    jobs = [(vfile, args, i % gpu_count) for i, vfile in enumerate(filelist)]

    print("Processing {} video files...".format(len(filelist)))

    p = ThreadPoolExecutor(max(1, args.ngpu))

    futures = [p.submit(mp_handler, j) for j in jobs]
    _ = [r.result() for r in tqdm(as_completed(futures), total=len(futures))]

    print('Dumping audios...')

    for vfile in tqdm(filelist):
        try:
            process_audio_file(vfile, args)
        except KeyboardInterrupt:
            exit(0)
        except:
            traceback.print_exc()
            continue

# Entry point
if __name__ == '__main__':
    main(args)


In [ ]:
nvidia-smi

conda activate echolip

pip install torch==2.2.0+cu122 torchvision==0.17.0+cu122 torchaudio==2.2.0+cu122 -f https://download.pytorch.org/whl/torch_stable.html
pip install -U tqdm

python preprocess.py --ngpu 4


